In [1]:
import torch
import numpy

from torch import tensor
from torch import nn
from torch import optim
from torch.autograd import Variable

from numpy import array

# Parameters

In [2]:
# Number of recognized words you put in input
nb_input = 3

# Number of classe, constant
nb_output = 6

# Number of hidden layers
nb_hidd_lay = 3

# RNN implementation

In [22]:
class RNN(nn.Module):
    def __init__(self, nb_inputs, nb_layers, nb_neurons, nb_outputs, learning_rate):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=nb_inputs, num_layers=nb_layers,
                   hidden_size=nb_neurons, dropout=0.5, batch_first=True)
        self.softmax = nn.Softmax(nb_outputs)
        
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.input_dim = nb_inputs
        self.output_dim = nb_output
        self.nb_layers = nb_layers
        self.nb_neurons = nb_neurons
        
    def forward(self, inputs):
        h0 = Variable(torch.zeros(self.nb_layers, inputs.size(0), self.nb_neurons))
        x, hn = self.rnn(inputs, h0)
        return self.softmax(x)
    
def learn(rnn, batch_list, num_epochs=1,):
    rnn.train()
    losses = []
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_idx, (data, target) in enumerate(batch_list):
            data, target = Variable(data), Variable(target)
            
            output = rnn(data)

            loss = criterion(output, target)

            losses.append(loss.data.item())

            rnn.optimizer.zero_grad()

            loss.backward()
            rnn.optimizer.step()

            if batch_idx % 100 == 0 or batch_idx % 100 == 1 :
                print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                        epoch, 
                        batch_idx * len(data), 
                        len(batch_list),
                        100. * batch_idx / len(batch_list), 
                        loss.data.item()), 
                        end='')

# Let's try our RNN

In [23]:
# Basically, meet a 1 and activate the first output; the second one if only 0 have been encountered
batch_list = [
    [tensor([[ [0., 0.], [0., 0.], [0., 0.] ]]), tensor([ [0, 1] ])],
    [tensor([[ [0., 0.], [0., 0.], [0., 0.], [0., 0.] ]]), tensor([ [0, 1] ])],
    [tensor([[ [1., 0.], [0., 0.], [0., 0.] ]]), tensor([ [1, 0] ])],
    [tensor([[ [0., 0.], [1., 0.], [0., 0.] ]]), tensor([ [1, 0] ])],
    [tensor([[ [0., 0.], [1., 0.], [1., 0.] ]]), tensor([ [1, 0] ])],
]

rnn = RNN(nb_inputs = 2, nb_layers=3, nb_neurons=2, nb_outputs=2, learning_rate=0.01)

learn(rnn, batch_list)

# Where I am now: whaaattt when did I tell you my batch size is 2 now? It only is 1!

 Train Epoch: 0 [1/5 (20%)]	 Loss: 1.326466